Read in a spreadsheet of data from sam 

In [ ]:
import os,sys,csv,string,json
from  timestamps import human2number,number2human

In [ ]:
files = ["494483.json","494864.json","495163.json"]

In [ ]:
configfile =files[2]
filename = configfile
if os.path.exists(configfile):
  with open(configfile,'r') as f:
    config = json.load(f)
else:
  print ("no config file",configfile)
  sys.exit(0)

print ("Top level",config.keys())
print ("Next level", config["hits"].keys())

the actual data is 2 layers down - get it and parse into info

In [4]:
data = config["hits"]["hits"]

print (data[0].keys())
info = {}
allkeys = ["kafka_timestamp"]

drop = ["key","type","@version","consumer_group"]

#   '@version': '1', '@timestamp': '2021-01-01T01:27:50.238Z', 'file_state': 'transferred', 'username': 'spurgeon'} 
ofile = open("log.txt",'w')
counter = 0
debug = False
limit = 10000
file_id = -1

fileurlmap = {}
filesizemap = {}
for line in data[0:limit]:
  counter +=1
  debug = counter < 10
  record = line["_source"]
  tag = ""
    
  if not "project_id" in record:
     #print ('no project_id', record)
     continue
  else:
    project_id = record["project_id"]
    
  if not "@timestamp" in record:
     #print ('no project_id', record)
     continue
  else:
    timestamp = human2number(record["@timestamp"])
    #print (record["@timestamp"],timestamp,number2human(timestamp))
    
  if "project_name" in record:
    if "prestage" in record["project_name"]:
        #print ("skip prestage",project_id)
        continue
    
  if type(project_id)!= type(1):
    print ("strange type",project_id,type(project_id),type(1))
    continue
  
  if not "file_id" in record:
     #print ('no file_id', record)
     continue
  else:
    file_id = record["file_id"]
  debug = file_id == 43409208

  if "file_name" in record:
    file_name = record["file_name"]
  else:
    file_name = None
    
  if "file_url" in record:
    fileurlmap[file_id] = record["file_url"]
  if "file_size" in record:
    filesizemap[file_id] = record["file_size"]
   
    
  #timestamp = record["kafka"]["timestamp"]
  
  #print ("project id",project_id,record)
  if project_id not in info:
     #print (" new project", project_id, file_id, timestamp)
     info[project_id] = {}
        
  if file_id not in info[project_id]:
     #print (" new file_id", project_id, file_id,timestamp)
     info[project_id][file_id] = {}
    
  info[project_id][file_id][timestamp] = {}      
 
  for field in record:
      if field not in drop:
          #print ("add field",project_id,file_id,timestamp,field,record[field])
          info[project_id][file_id][timestamp][field]=record[field]
      if field not in allkeys:
        allkeys.append(field)
        
  if not "file_name" in info[project_id][file_id][timestamp] and file_name != None:
        info[project_id][file_id][timestamp]["file_name"]= file_name
        
  if file_id in fileurlmap and fileurlmap[file_id] != None:
        info[project_id][file_id][timestamp]["file_url"]= fileurlmap[file_id]
  if file_id in filesizemap and filesizemap[file_id] != None:
        info[project_id][file_id][timestamp]["file_size"]= filesizemap[file_id]
        
  #info[project_id][file_id][timestamp]["kafka_timestamp"] = timestamp
   
  s = "record %s %s %s %s \n"%(project_id, file_id,timestamp,info[project_id][file_id][timestamp])
  if (debug):
        print (s)
  #print ("new record",project_id, process_id,timestamp)
  ofile.write(s)
 # else:
  #    info[process_id][timestamp]["kafka_"+field]=record[field]
  #print("info",info[timestamp])

counter = 0

# now dump to a csv file
csv_file = "inputs_"+filename+".csv"

try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=allkeys)
        writer.writeheader()
        
        for project in info:
            for file_id in info[project]:
                for timestamp in info[project][file_id]:
                    writer.writerow(info[project][file_id][timestamp])         
except IOError:
    print("I/O error")
    
# for item in info.keys()[0].
#     counter += 1
#     if counter > 2:
#         continue
#     for timestamp in info[item]:
#         print ("series",item,timestamp,info[item][timestamp])
ofile.close()

dict_keys(['_index', '_type', '_id', '_score', '_source'])


In [5]:
   
    
# now merge records for each jobid into a single record

single = []

keys = None
counter = 0
for project_id in info:
  for file_id in info[project_id]:
    file = info[project_id][file_id]
    debug = counter < 2
#     if debug:
#         print ("process", process)
    stamps = list(file.keys())
    if len(stamps)<2:
        print ("only one relevant event")
        continue
     
    print ("file keys",stamps,file_id)
    file_url = ""
    ended  = False
    file_size = 0
    #kafka_begin = 0
    
    # need to sort them by time and then pick out more recent data
    ended = False
    file_state = "Unknown"
    process_state = "Unknown"
    project = "Unknown"
    user = "Unknown"
    started = False
    node = "Unknown"
    #kafka_begin = 0
    #kafka_end = 0
    success = False
    firstfile = 0
    process_id = 0
    file_count = 0
    for i in sorted(stamps):
        
        # info from all records
        lasttime = i
        record = file[i]
        
#         if debug:
        print ("record",i, record)
        timestamp = i
        
        event = record["event"]
        if "project_name" in record:
            project = record["project_name"]
        if "file_id" in record and record["file_id"] != file_id:
            print ( " strange record file_id",file_id, record["file_id"])
        if not started:
            start_time=timestamp
            started = True;
            print (" Start of file")
        
        # update information 
        if "file_state" in record:
            file_state = record["file_state"]
        if "process_state" in record:
            process_state = record["process_state"]
        if "file_id" in record:
            file_id = record["file_id"]
            if "file_size" in record:
                file_size = record["file_size"]
        if "event" in record:
            event = record["event"]
            
        #print ("state",project_id,process_id,i,file_id,file_state,process_state,event)   
        if file_id != firstfile and firstfile != 0:
            print ("found a new filename should have quit already",file_id,first_file)
            ended = True
        #info just from the transfers not interested in prep 
        if file_state not in ["consumed","skipped"] and process_state not in ["consumed_file","finished"]:
            continue
        if "process_id" in record:
            process_id =record["process_id"]
        #if kafka_begin == 0:
        #    kafka_begin = record["kafka_timestamp"]
        #kafka_end = record["kafka_timestamp"]
            
        if "file_url" in record:
            file_url = record["file_url"]
        if "node" in record:
            node = record["node"]
        if "username" in record:
            user = record["username"]
        
        if event in ["consumed_file"]:
            end_time = timestamp
            ended = True
            success = True
            print ("consumed file event")
            #break
            
        if file_state == "transferred":
            start_time = timestamp
        if process_state in ["completed","finished"] :
            end_time = timestamp
            success = True
            ended = True
            print ("process completed")
            #break 
        if file_state in ["skipped"]:
            end_time = timestamp
            ended = True
            success = False
            print ("file skipped")
            #break
        if file_state == "consumed":
            end_time = timestamp
            success = True
            ended = True
            print ("file consumed")
            # don't break in case we can catch a finished step
        
            
        print ("step",project_id,process_id,file_id,file_size,start_time,i,event,file_state,process_state)

        if not ended: 
            end_time = lasttime
            continue
        
        # make the records out of the bits you collected
        new = {}
        new["file_id"]=file_id
        new["start_time"] = start_time
        new["end_time"]= end_time
        new["last_event"]=event
        new["project_name"]=project
        new["node"]=node
        new["file_id"]=file_id
        new["file_url"]=file_url
        new["file_size"]=file_size
        new["user"]=user
        new["last_file_state"]=file_state
        new["last_process_state"]=process_state
        new["ended"]=ended
        #new["kafka_begin"]= kafka_begin
        #new["kafka_end"]= kafka_end
        new["livetime"]=(end_time-start_time)
        new["Success"]=success
        new["username"]=user
        new["project_id"]=project_id
        new["file_count"]=file_count
        new["rate"]=file_size/new["livetime"]*0.000001

        if file_size > 0:
            single.append(new)
        
        file_id = 0
        file_count = 0 
        ended = False
        firstfile = 0
        success = False
        last_event = ""
        end_time = end_time
   
#print (single[0])
keys = single[0].keys()        
        
# now dump to a csv file
csv_file = "processed_"+filename+".csv"

try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=keys)
        writer.writeheader()
        
        for data in single:
           
            writer.writerow(data)
            
except IOError:
    print("I/O error")
    

    

only one relevant event
file keys [1612880298.913, 1612880298.317, 1612880549.977] 35553547
record 1612880298.317 {'file_url': 'root://fndca1.fnal.gov:1096/pnfs/fnal.gov/usr/dune/tape_backed/dunepro/physics/full-reconstructed/2019/mc/out1/PDSPProd2/22/67/19/16/PDSPProd2_protoDUNE_sp_reco_35ms_sce_off_23536159_0_555b67d8-206c-4d55-8ebd-b9fda7e359f2.root', 'project_name': 'sbjones_mc_1GeV_full_reconstructed_prod2_sce_off_20210209075809', 'process_state': 'active', 'experiment': 'dune', 'project_id': 495163, 'file_size': 1878172981, 'snapshot_id': 375522, 'job_id': '16914329.0@jobsub03.fnal.gov', 'kafka': {'topic': 'ingest.sam.events', 'partition': 0, 'consumer_group': 'ingest_sam_logstash_prod', 'timestamp': 1612880298317, 'key': None, 'offset': 8220507907}, 'process_id': 14825604, 'event': 'open_file', 'file_id': 35553547, 'node': 'b7s13n1823.cern.ch', 'station': 'dune', 'event_time': '2021-02-09T08:18:18.317205-06:00', 'file_name': 'PDSPProd2_protoDUNE_sp_reco_35ms_sce_off_23536159_0_5

ZeroDivisionError: float division by zero